<a href="https://colab.research.google.com/github/JasonArmitage-res/Lab_Multimodal_ML/blob/master/ImageUnimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
!pip install --upgrade tables

dataset_embeddings = pd.read_hdf('/content/drive/My Drive/dataset/mm_imdb_embeddings.h5', 'embeddings')

In [0]:
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split


def Train_Test_Split(data , test_data_fraction = 0.2) :
    
    mlb = MultiLabelBinarizer()
    data_genres_one_hot_encoding = mlb.fit_transform(data['genres'])
    Label_names = mlb.classes_
    data_genres_one_hot_encoding = pd.DataFrame(data_genres_one_hot_encoding, columns = mlb.classes_)
    Data_train, Data_test, Labels_train, Labels_test = train_test_split(data, data_genres_one_hot_encoding, test_size = test_data_fraction)
    Labels_train = torch.tensor(Labels_train.values)
    Labels_test = torch.tensor(Labels_test.values)
    
    Data_train = Data_train.reset_index(drop=True)
    Data_test = Data_test.reset_index(drop=True)

    return (Data_train, Data_test, Labels_train, Labels_test, Label_names)
    
Data_train, Data_test, Labels_train_tensor, Labels_test_tensor, Label_names = Train_Test_Split(dataset_embeddings)

In [0]:
#For image unimodal classifier
import numpy as np

Data_train_tensor = torch.tensor(Data_train['vgg16_embeddings'])
Data_test_tensor = torch.tensor(Data_test['vgg16_embeddings'])

In [0]:
import torch
from torch import nn
from torchvision import models, transforms

class VGG16MultiLabelClassifier(nn.Module):
  
  def __init__(self, hidden_layer_size = 512, hidden_activation = "tanh", input_size = 4096, num_labels = 23, dropout = 0.1, gen_embeddings = False):

        super(VGG16MultiLabelClassifier,self).__init__()
        self.num_labels = num_labels
        self.input_size = input_size
        self.base_model = models.vgg16()
        self.hidden_layer_size = hidden_layer_size
        self.gen_embeddings = gen_embeddings
        self.embedding_layers = list(self.base_model.classifier.children())[:-2] 
        self.base_model.classifier = nn.Sequential(*self.embedding_layers)

        self.hidden_layer = torch.nn.Linear(self.input_size, self.hidden_layer_size)

        if(hidden_activation == "tanh") :
          self.hidden_activation = torch.nn.Tanh()
        elif(hidden_activation == "relu") :
          self.hidden_activation = torch.nn.ReLU()
        elif(hidden_activation == "sigmoid") :
          self.hidden_activation = torch.nn.Sigmoid()
        elif(hidden_activation == "leaky_relu") :
          self.hidden_activation = torch.nn.LeakyReLU()
        else :
          return("Invalid hidden_activation parameter value.")

        self.output_layer = torch.nn.Linear(self.hidden_layer_size, self.num_labels)
        self.output_activation = torch.nn.Sigmoid()
        self.dropout = nn.Dropout(p=dropout)


  def forward(self, input):
        
        if(self.gen_embeddings):
            embeddings = self.base_model(input)
            return embeddings
        else :
            embeddings = input
        
        logits1 = self.hidden_layer(embeddings)
        activation1 = self.hidden_activation(logits1)

        dropped = self.dropout(activation1)

        logits2 = self.output_layer(dropped)
        if(self.training) :
            return logits2
        else :
            output = self.output_activation(logits2)
            return output


  def freeze_base_model(self):
        for param in self.base_model.features.parameters():
            param.requires_grad = False


  def unfreeze_base_model(self):
        for param in self.base_model.features.parameters():
            param.requires_grad = True
        

In [0]:
import torch
from torch.utils.data import DataLoader, SequentialSampler

def GetImageEmbeddings_VGG16(dataset, batch_size = 32):
  
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  image_tensors = ImageTransformation_VGG16(dataset['image'])
  sampler = SequentialSampler(image_tensors)
  dataloader = DataLoader(image_tensors, sampler=sampler, batch_size=batch_size)
  
  results = torch.Tensor().to(device)

  model = VGG16MultiLabelClassifier(gen_embeddings = True)
  model.to(device)
  model.eval()

  for num, batch_data in enumerate(dataloader): 
      print(num)   
      data = batch_data.to(device)
      with torch.no_grad():
        emdeddings = model(data)
      results = torch.cat((results, emdeddings))

  return results


In [0]:
from torch.utils.data import Dataset

class ImageTransformation_VGG16(Dataset) :

    def __init__(self, PIL_images_list, train = True):

        self.PIL_images_list = PIL_images_list
        self.train = train

        self.transformations_train = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

        self.transformations_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def __len__(self):
        return len(self.PIL_images_list)

    def __getitem__(self, idx):

        if self.train:
            sample = self.transformations_train(self.PIL_images_list[idx])
        else:
            sample = self.transformations_test(self.PIL_images_list[idx])
        return sample

In [0]:
import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange
!pip install transformers
from transformers import BertConfig, AdamW, get_linear_schedule_with_warmup
from sklearn import metrics
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
import pandas as pd
import time
import datetime

'''
USAGE :

1. If text_unimodal = True
   Data_train_tensor , Data_test_tensor : Image embeddings as tensors of shape [#train_samples, 4096], [#test_samples, 4096]

2. If image_unimodal = True
   Data_train_tensor , Data_test_tensor : Text embeddings as tensors of shape [#train_samples, 768], [#test_samples, 768]

3. If multimodal = True (default)
   Data_train_tensor , Data_test_tensor : 

4. Labels_train_tensor, Labels_test_tensor : one-hot encoded labels as tensors of shape [#train_samples, 23], [#test_samples, 23]

'''

class Training_Testing():

    def __init__(self, Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, hidden_layer_size = 512,
                 Label_names = None, freeze_base_model = True, image_unimodal = False, text_unimodal = False,
                 hidden_activation = "tanh", batch_size = 32, epochs = 10, sigmoid_thresh = 0.2, learning_rate = 2e-5, num_labels = 23, dropout = 0.1):

      
      #self.dropout = dropout
      #self.hidden_layer_size = hidden_layer_size
      #self.hidden_activation = hidden_activation

      if(text_unimodal) :
        #self.config = BertConfig.from_pretrained('bert-base-uncased')
        self.model = BertMultiLabelClassifier(hidden_layer_size = hidden_layer_size, hidden_activation = hidden_activation, dropout = dropout).cuda()

      
      elif(image_unimodal) :
        self.model = VGG16MultiLabelClassifier(hidden_layer_size = hidden_layer_size, hidden_activation = hidden_activation, dropout = dropout).cuda()
 

      if(freeze_base_model) :
        self.model.freeze_base_model()
      self.label_names = Label_names
      self.num_labels = num_labels
      self.batch_size = batch_size
      self.learning_rate = learning_rate
      self.epochs = epochs
      self.sigmoid_thresh = sigmoid_thresh
      self.optimizer = self.SetOptimizer()
      self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      self.results = pd.DataFrame(0, index=['Recall','Precision','F_Score'], columns=['micro', 'macro', 'weighted', 'samples']).astype(float)
      self.epoch_loss_set = []
      self.train_dataloader = self.SetTrainDataloader(Data_train_tensor, Labels_train_tensor)
      self.test_dataloader = self.SetTestDataloader(Data_test_tensor, Labels_test_tensor)
      self.scheduler = self.SetScheduler()


    def SetOptimizer(self) :

      optimizer = AdamW(self.model.parameters(), self.learning_rate, eps = 1e-6)
      return(optimizer)

    

    def SetScheduler(self) :

      scheduler = get_linear_schedule_with_warmup(self.optimizer, num_warmup_steps = 0, 
                                                  num_training_steps = self.epochs*len(self.train_dataloader))
      return(scheduler) 



    def Get_Metrics(self, actual, predicted) :

      #acc = metrics.accuracy_score(actual, predicted)
      #hamming = metrics.hamming_loss(actual, predicted)
      #(metrics.roc_auc_score(actual, predicted, average=average)
      averages = ('micro', 'macro', 'weighted', 'samples')
      for average in averages:
          precision, recall, fscore, _ = metrics.precision_recall_fscore_support(actual, predicted, average=average)
          self.results[average]['Recall'] += recall
          self.results[average]['Precision'] += precision
          self.results[average]['F_Score'] += fscore


    #source: https://mccormickml.com/2019/07/22/BERT-fine-tuning/
    def Plot_Training_Epoch_Loss(self) :

      sns.set(style='darkgrid')
      sns.set(font_scale=1.5)
      plt.rcParams["figure.figsize"] = (12,6)
      plt.plot(self.epoch_loss_set, 'b-o')
      plt.title("Training loss")
      plt.xlabel("Epoch")
      plt.ylabel("Loss")
      plt.savefig('Training_Epoch_Loss.png',bbox_inches='tight')
      plt.show()


    #source: https://mccormickml.com/2019/07/22/BERT-fine-tuning/
    def format_time(self, elapsed):
      '''
      Takes a time in seconds and returns a string hh:mm:ss
      '''
      # Round to the nearest second.
      elapsed_rounded = int(round((elapsed)))
      return str(datetime.timedelta(seconds=elapsed_rounded))


    def SetTrainDataloader(self, Data_train_tensor, Labels_train_tensor) :

      train_dataset = TensorDataset(Data_train_tensor, Labels_train_tensor)
      train_sampler = RandomSampler(train_dataset)
      train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size = self.batch_size)
      return(train_dataloader)


    def SetTestDataloader(self, Data_test_tensor, Labels_test_tensor) :
      
      test_dataset = TensorDataset(Data_test_tensor, Labels_test_tensor)
      test_sampler = SequentialSampler(test_dataset)
      test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size = Data_test_tensor.shape[0])
      return(test_dataloader)


    def Train(self) :

      for _ in trange(self.epochs, desc="Epoch"):
        
        self.model.train()
        epoch_loss = 0

        # Measure how long the training epoch takes.
        t0 = time.time()
    
        for step_num, batch_data in enumerate(self.train_dataloader):

          # Progress update every 30 batches.
          if step_num % 30 == 0 and not step_num == 0:
            elapsed = self.format_time(time.time() - t0)
            print('  Batch : ',step_num, ' , Time elapsed : ',elapsed)

          samples, labels = tuple(t.to(self.device) for t in batch_data)
          self.optimizer.zero_grad()
          logits = self.model(samples.float())
          loss_fct = BCEWithLogitsLoss()
          batch_loss = loss_fct(logits.view(-1, self.num_labels).float(), labels.view(-1, self.num_labels).float())
          batch_loss.backward()
          self.optimizer.step()
          self.scheduler.step()
          epoch_loss += batch_loss.item()

        avg_epoch_loss = epoch_loss/len(self.train_dataloader)
        print("\nTrain loss for epoch: ",avg_epoch_loss)
        print("\nTraining epoch took: {:}".format(self.format_time(time.time() - t0)))
        self.epoch_loss_set.append(avg_epoch_loss)

      torch.save(self.model.state_dict(), "/content/drive/My Drive/dataset/model.pt")
      self.Plot_Training_Epoch_Loss()
    

    def Test(self) :

      # Put model in evaluation mode to evaluate loss on the test set
      self.model.eval()

      for batch_data in self.test_dataloader:
  
        samples, labels = tuple(t.to(self.device) for t in batch_data)
      
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        # Forward pass, calculate logit predictions
        with torch.no_grad():
          output = self.model(samples)

        threshold = torch.Tensor([self.sigmoid_thresh]).to(self.device)
        predictions = (output > threshold).int()

        # Move preds and labels to CPU
        predictions = predictions.detach().cpu().numpy()
        labels = labels.to('cpu').numpy()
      
        self.Get_Metrics(labels, predictions)
    
      self.results = self.results/len(self.test_dataloader)
      #print("Test data metrics : \n")
      
      return(self.results)

    

In [0]:
train_test = Training_Testing(Data_train_tensor, Labels_train_tensor, Data_test_tensor, Labels_test_tensor, Label_names = Label_names
                              hidden_layer_size=500, hidden_activation = "relu", image_unimodal = True, epochs = 100, batch_size= 256, learning_rate=0.001, dropout=0.5, sigmoid_thresh = 0.2)
train_test.Train()
train_test.Test()